In [ ]:
from lstm_models import LSTM
from gpu_dataloader import GPUDataset
from torch.utils.data import DataLoader, SubsetRandomSampler
import torch.nn as nn
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# plotting the data
import matplotlib.pyplot as plt
# used for the dataframes
import pandas as pd


import numpy as np
%matplotlib inline

from utils import get_device_as_string, get_device

In [ ]:
import math

def get_rmse(actual_values, predicted_values) -> float:
    '''returns the root mean squared error'''
    return math.sqrt(mean_squared_error(actual_values, predicted_values))

def get_mape(actual_values, predicted_values):
    '''returns the mean absolue percentage error'''
    return np.mean(np.abs(actual_values - predicted_values) / np.abs(actual_values) * 100)

def get_mae(actual_values, predicted_values) -> float:
    '''returns the mean absolute error'''
    return mean_absolute_error(actual_values, predicted_values)

In [ ]:
batch_size: int = 200
dataset = GPUDataset(small_df=True, batch_size=batch_size)

In [ ]:
num_epochs: int = 5
learning_rate: float = 0.01

# number of features
input_size: int = dataset.X.shape[2]
# number of features in hidden state
hidden_size: int = dataset.X.shape[2] * 64
# number of stacked lstm layers
num_layers: int = 1
# number of output classes
num_classes: int = dataset.y.shape[1]

device = get_device()

In [ ]:
import wandb
wandb.init(project='Hardware Utilization Prediction')

wandb.config.num_epochs = num_epochs
wandb.config.learning_rate = learning_rate
wandb.config.input_size = input_size
wandb.config.hidden_size = hidden_size
wandb.config.num_layers = num_layers
wandb.config.num_classes = num_classes

In [ ]:
lstm = LSTM(num_classes, input_size, hidden_size, num_layers, dataset.X.shape[1])

In [ ]:
LOSS: str = 'loss'
RMSE_TRAINING: str = 'root mean squared error (training)'
MAE_TRAINING: str = 'mean absolute error (training)'

wandb.define_metric(LOSS, summary='min')
wandb.define_metric(RMSE_TRAINING, summary='min')
wandb.define_metric(MAE_TRAINING, summary='min')

In [ ]:
# lstm = lstm.to(device)
lstm.train()
# log gradients and model parameters
wandb.watch(lstm)

lstm


In [ ]:
# mean squared error for regression
criterion = nn.MSELoss()
criterion = criterion.to(get_device())
# optimizer function
optimizer = torch.optim.AdamW(lstm.parameters(), lr=learning_rate)

In [ ]:
import random

from utils import get_device

for epoch in range(num_epochs):

    batch_section = [idx for idx in range(len(dataset) // batch_size)]

    while len(batch_section) > 0:
        choice = random.choice(batch_section)
        sampler = SubsetRandomSampler(
            list(range(choice*batch_size, (choice+1)*batch_size)))
        train_loader = DataLoader(
            dataset, batch_size=batch_size, shuffle=False, num_workers=6, sampler=sampler)

        for _, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()

            outputs = lstm.forward(inputs)
            outputs = outputs.to(device)

            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            wandb.log({'loss batch': loss.item()})

        # remove batch section after training on it
        batch_section.remove(choice)
        

    # logging to wandb
    if get_device_as_string() == 'cuda':
        o = outputs.cpu().detach().numpy()
        l = labels.cpu().detach().numpy()
    else:
        o = outputs.detach().numpy()
        l = labels.detach().numpy()
    rmse = get_rmse(o, l)
    mae = get_mae(o, l)
    log_dict: dict = {
        LOSS: loss.item(),
        RMSE_TRAINING: rmse,
        MAE_TRAINING: mae,
    }
    wandb.log(log_dict)

    print(f'Epoch: {epoch}, loss: {loss.item():2f}, rmse: {rmse:2f}, mae: {mae:2f}')


In [ ]:
import time

current_time = time.ctime()
current_time

## Save the Model to Disk

In [ ]:
# torch.save(lstm.state_dict(), f'models/epochs-{num_epochs}-{current_time}')

In [ ]:
lstm.eval()

X_df = dataset.X
X_df = X_df.to(device)
# forward pass
prediction = lstm(X_df)
if get_device_as_string() == 'cuda':
    prediction = prediction.cpu().data.numpy()
else:
    prediction = prediction.data.numpy()
prediction = dataset.standard_scaler.fit_transform(prediction)

actual_data = dataset.y.data.numpy()
actual_data = dataset.minmax_scaler.fit_transform(actual_data)

# reverse transformation
prediction = dataset.standard_scaler.inverse_transform(prediction)
actual_data = dataset.minmax_scaler.inverse_transform(actual_data)

label_columns = dataset.get_default_label_columns()
# create dataframes
prediction_df = pd.DataFrame(prediction, columns=label_columns)
actual_data_df = pd.DataFrame(actual_data, columns=label_columns)


## Calculate Root Mean Squared Error

Calculating the RMSE for the overall prediction of the (training) dataset.

In [ ]:
rmse_key: str = 'Root Mean Squared Error (Overall - Training)'
rmse_result = get_rmse(actual_data_df[:], prediction_df[:])
print(f'Test Score: {rmse_result:.2f} RMSE')
wandb.summary[rmse_key] = rmse_result

## Calculate Mean Absolute Error

Calcutlate the MAE for the overall prediction of the (training) dataset.

In [ ]:
mae_key: str = 'Mean Absolute Error (Overall - Training)'
mae_result = mean_absolute_error(actual_data_df[:], prediction_df[:])
print(f'Test Score: {mae_result} MAE')
wandb.summary[mae_key] = mae_result

In [ ]:
def plot_column(actual_values=actual_data_df, predicted_values=prediction_df, column_number: int = 0, rmse_threshold: float = 0.30, is_training: bool = True):

    if len(label_columns) <= column_number:
        print('Out of Prediction Bounds')
        return

    plt.figure(figsize=(25, 15))  # plotting

    column = label_columns[column_number]
    pred_column = f"pred_{column}_{'training' if is_training else 'test'}"

    rmse = get_rmse(actual_values[column], predicted_values[column])
    mae = mean_absolute_error(actual_values[column], predicted_values[column])

    predicted_color = 'green' if rmse < rmse_threshold else 'orange'

    plt.plot(actual_values[column], label=column, color='black')  # actual plot
    plt.plot(predicted_values[column], label='pred_' +
             column, color=predicted_color)  # predicted plot

    plt.title('Time-Series Prediction')
    plt.plot([], [], ' ', label=f'RMSE: {rmse}')
    plt.plot([], [], ' ', label=f'MAE: {mae}')
    plt.legend()
    
    wandb.log({pred_column: wandb.Image(plt)})
    wandb.summary[f'Root Mean Squared Error ({column})'] = rmse
    wandb.summary[f'Mean Absolute Error ({column})'] = mae
    
    plt.show()


## See Predictions on Training Dataset

In [ ]:
plot_column(column_number=0)

In [ ]:
plot_column(column_number=1)

In [ ]:
plot_column(column_number=2)

In [ ]:
plot_column(column_number=3)

In [ ]:
plot_column(column_number=4)

In [ ]:
plot_column(column_number=5)

In [ ]:
plot_column(column_number=6)

## Test Set Analysis

Below, the test set will be loaded and the model evaluated with it to see the actual performance.

In [ ]:
test_dataset = GPUDataset(is_training=False, small_df=True, batch_size=batch_size)

In [ ]:
lstm.eval()

X_df = test_dataset.X
X_df = X_df.to(device)
# forward pass
prediction = lstm(X_df)
if get_device_as_string() == 'cuda':
    prediction = prediction.cpu().data.numpy()
else:
    prediction = prediction.data.numpy()
prediction = test_dataset.standard_scaler.fit_transform(prediction)

actual_data = test_dataset.y.data.numpy()
actual_data = test_dataset.minmax_scaler.fit_transform(actual_data)

# reverse transformation
prediction = test_dataset.standard_scaler.inverse_transform(prediction)
actual_data = test_dataset.minmax_scaler.inverse_transform(actual_data)

label_columns = test_dataset.get_default_label_columns()
# create dataframes
prediction_df = pd.DataFrame(prediction, columns=label_columns)
actual_data_df = pd.DataFrame(actual_data, columns=label_columns)


In [ ]:
rmse_key: str = 'Root Mean Squared Error (Overall - Test)'
rmse_result = get_rmse(actual_data_df[:], prediction_df[:])
print(f'Test Score: {rmse_result:.2f} RMSE')
wandb.summary[rmse_key] = rmse_result

In [ ]:
mae_key: str = 'Mean Absolute Error (Overall - Test)'
mae_result = mean_absolute_error(actual_data_df[:], prediction_df[:])
print(f'Test Score: {mae_result} MAE')
wandb.summary[mae_key] = mae_result

In [ ]:
plot_column(column_number=0)

In [ ]:
plot_column(column_number=1)

In [ ]:
plot_column(column_number=2)